In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.functions import *

In [0]:
df = spark.read.load('/FileStore/tables/googleplaystore.csv',format="csv",sep=',',header=True,inferSchema=True,escape = '"')
df.show(5,5)

+-----+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|  App|Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+-----+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|Ph...|   AR...|   4.1|    159| 19M|   10...|Free|    0|         Ev...| Ar...|       Ja...|      1.0.0|      4....|
|Co...|   AR...|   3.9|    967| 14M|   50...|Free|    0|         Ev...| Ar...|       Ja...|      2.0.0|      4....|
|U ...|   AR...|   4.7|  87510|8.7M|   5,...|Free|    0|         Ev...| Ar...|       Au...|      1.2.4|      4....|
|Sk...|   AR...|   4.5|  21...| 25M|   50...|Free|    0|          Teen| Ar...|       Ju...|      Va...|      4....|
|Pi...|   AR...|   4.3|    967|2.8M|   10...|Free|    0|         Ev...| Ar...|       Ju...|        1.1|      4....|
+-----+--------+------+-------+----+--------+----+-----+--------------+-

In [0]:
df.count()

Out[3]: 10841

In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [0]:
df.show(1,1)

+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|App|Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|  P|       A|     4|      1|   1|       1|   F|    0|             E|     A|           J|          1|          4|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
only showing top 1 row



In [0]:
df = df.drop("Size","Content Rating","Last Updated","Android Ver","Current Ver")
df.show(1)

+--------------------+--------------+------+-------+--------+----+-----+------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|      Genres|
+--------------------+--------------+------+-------+--------+----+-----+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|Art & Design|
+--------------------+--------------+------+-------+--------+----+-----+------------+
only showing top 1 row



In [0]:
from pyspark.sql.functions import col,regexp_replace
df = df.withColumn("Reviews",col("Reviews").cast(IntegerType()))\
    .withColumn("Installs",regexp_replace(col("Installs"),"[^0-9]",""))\
    .withColumn("Installs",col("Installs").cast(IntegerType()))\
    .withColumn("Price",regexp_replace(col("Price"),"[$]",""))\
    .withColumn("Price",col("Price").cast(IntegerType()))
df.show(1)
df.printSchema()

+--------------------+--------------+------+-------+--------+----+-----+------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|      Genres|
+--------------------+--------------+------+-------+--------+----+-----+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|Art & Design|
+--------------------+--------------+------+-------+--------+----+-----+------------+
only showing top 1 row

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



In [0]:
#replace 0 with Null
#df = df.replace(0,None,subset = ['Price'])
#replace NAN with 0
df = df.fillna(0,subset =['Rating'])


In [0]:
df.createOrReplaceTempView("apps")

In [0]:
%sql
select * from apps limit 1

App,Category,Rating,Reviews,Installs,Type,Price,Genres
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,10000,Free,0,Art & Design


In [0]:
%sql
select App,sum(Reviews) as Reviews from apps 
group by App
order by Reviews desc
limit 10

App,Reviews
Instagram,266241989
WhatsApp Messenger,207348304
Clash of Clans,179558781
Messenger – Text and Video Chat for Free,169932272
Subway Surfers,166331958
Candy Crush Saga,156993136
Facebook,156286514
8 Ball Pool,99386198
Clash Royale,92530298
Snapchat,68045010


In [0]:
%sql
---select App,Type,Installs from apps

(select App,Type,sum(Installs) as Installs from apps
where Type = 'Free'
group by  App,Type
order by Installs desc
limit 10)
union
(select App,Type,sum(Installs) as Installs from apps
where Type = 'Paid'
group by  App,Type
order by Installs desc
limit 10)

App,Type,Installs
Gmail,Free,3000000000
Candy Crush Saga,Free,3500000000
Subway Surfers,Free,6000000000
Instagram,Free,4000000000
Google Drive,Free,4000000000
Hangouts,Free,4000000000
Google News,Free,4000000000
Temple Run 2,Free,3000000000
WhatsApp Messenger,Free,3000000000
Google Photos,Free,4000000000


In [0]:
#fetch unique elements from column
df1 = df.select(col("Installs")).distinct()
df1.show()



+----------+
|  Installs|
+----------+
|    500000|
|    100000|
|      null|
|         1|
|      5000|
|       500|
|     10000|
|         5|
|     50000|
|   5000000|
|  50000000|
|       100|
|      1000|
|   1000000|
|        10|
|        50|
| 500000000|
|1000000000|
|  10000000|
|         0|
+----------+
only showing top 20 rows



In [0]:
%sql
select Category,sum(Installs) as Installs
from apps
group by Category
order by Installs desc

Category,Installs
GAME,35086024415
COMMUNICATION,32647276251
PRODUCTIVITY,14176091369
SOCIAL,14069867902
TOOLS,11452771915
FAMILY,10258263505
PHOTOGRAPHY,10088247655
NEWS_AND_MAGAZINES,7496317760
TRAVEL_AND_LOCAL,6868887146
VIDEO_PLAYERS,6222002720


In [0]:
df1 = df.select(col("Rating")).distinct()
df1.show()

+------+
|Rating|
+------+
|   0.0|
|   3.5|
|   3.7|
|   4.5|
|   3.4|
|   4.9|
|   3.1|
|   4.1|
|   4.0|
|   3.9|
|   3.8|
|   4.2|
|   4.3|
|   4.7|
|   3.2|
|   4.4|
|   4.8|
|   4.6|
|   3.6|
|   5.0|
+------+
only showing top 20 rows



In [0]:
%sql
select App,sum(Price) as Price  
from apps
where Type ='Paid'
group by App
order by Price desc
limit 10 

App,Price
I'm Rich - Trump Edition,400
most expensive app (H),399
I am rich(premium),399
I Am Rich Premium,399
I am Rich Plus,399
I'm Rich/Eu sou Rico/أنا غني/我很有錢,399
I Am Rich Pro,399
I AM RICH PRO PLUS,399
I am Rich,399
I am Rich!,399


In [0]:
%sql

select distinct App,sum(Price) as Price,cast(avg(Rating) as decimal(10,1))  as Rating
from apps
where Type ='Paid' and Price > 0
group by App
order by Rating desc
limit 10

App,Price,Rating
30WPM Amateur ham radio Koch CW Morse code trainer,1,5.0
USMLE Step 2 CK Flashcards,19,5.0
Morse Player,1,5.0
AP Art History Flashcards,29,5.0
Ra Ga Ba,1,5.0
FHR 5-Tier 2.0,2,5.0
TI-84 CE Graphing Calculator Manual TI 84,4,5.0
Santa's Monster Shootout DX,1,5.0
BC MVA Fines,1,5.0
ADS-B Driver,1,5.0
